# Segragate file
- Blast only allow limited size
- Thus seperate it to several files

In [ ]:
import re
filename = "LWW016_all.fas"
path = "data\\fasta_file\\LWW016_all.fas"

def read_fas_as_list(path):
    pattern = r"[>][^>]+"
    with open (path,"r") as f:
        whole_data = f.read()
        return re.findall(pattern,whole_data)

def part_folder (num = 50000):
    datalist = read_fas_as_list(path)
    count = len(datalist[0])
    item_index = 0
    folder_num = 1
    temptxt = ""

    while item_index < len(datalist):
        count += len(datalist[item_index])
        if count < num:
            temptxt += datalist[item_index]
            item_index += 1
        else:
            newpath = "data\\fasta_file\\output\\" + filename[:-4] + "_" + str(folder_num) + ".fas"
            with open (newpath, "w") as f:
                f.write(temptxt)
            folder_num += 1
            count = len(datalist[item_index])
            temptxt = datalist[item_index]
            item_index += 1

    newpath = "data\\fasta_file\\output\\" + filename[:-4] + "_" + str(folder_num) + ".fas"
    with open (newpath, "w") as f:
                f.write(temptxt)

part_folder ()

# Blastn download as json
- parse json

In [ ]:
import json
import pandas as pd

file_name = "data\\blast_result\\blastoutput.json"

query = []
accesssion = []
identity = []
sciname = []
title = []


with open(file_name) as obj:
    getDatas = json.load(obj)

for getData in getDatas:
    for item in getDatas[getData]:
        query.append((item['report']['results']['search']['query_title']))
        accesssion.append(item['report']['results']['search']['hits'][0]['description'][0]['accession'])
        sciname.append(item['report']['results']['search']['hits'][0]['description'][0]['sciname'])
        title.append(item['report']['results']['search']['hits'][0]['description'][0]['title'])
        identity.append(item['report']['results']['search']['hits'][0]["hsps"][0]['identity']/10)


df = pd.DataFrame([query, accesssion, identity, sciname, title]).transpose()
df.columns = ['query', 'accesssion', 'identity', 'sciname', 'title']
df.to_csv("data\\blast_result\\output\\output.csv",index = False)

In [2]:
#Blastn

from Bio.Blast import NCBIWWW

path = "data\\fasta_file\\output\\LWW016_all_1.fas"
outpath = "data\\blast_result\\blast_output.xml"

def Blast_fasta(path,outpath):

    with open (path) as obj:
        fasta_file = obj.read()

    blast_result = NCBIWWW.qblast("blastn", "nt", fasta_file, megablast=True, hitlist_size=10)
    out = blast_result.read()

    with open(outpath, "w") as output_xml:
        output_xml.write(out)

    blast_result.close()


In [2]:
import os

filelist = os.listdir("data\\fasta_file\\output")
for item in filelist:
    path = "data\\fasta_file\\output\\" + item
    outpath = "data\\blast_result\\output\\" + item[:-4]+ "_out.xml"
    Blast_fasta(path,outpath)

In [1]:
# import os

# filelist = os.listdir("data\\fasta_file\\output")

# def work(item):
#     path = "data\\fasta_file\\output\\" + item
#     outpath = "data\\blast_result\\output\\" + item[:-4]+ "_out.xml"
#     Blast_fasta(path,outpath)

# import tqdm
# from multiprocessing import Pool
# with Pool(1) as p:
#     list(tqdm.tqdm(p.imap(work, filelist), total=len(filelist)))

  0%|          | 0/138 [00:00<?, ?it/s]

In [3]:
#Blastn

import re
outfilelist = []
outfile = os.listdir("data\\blast_result\\output")
pattern = r"[_][0-9]+"
for item in outfile:
    result = re.search(pattern,item)
    if result:
        outfilelist.append(item[result.start():result.end()])

templist = []
import os
filelist = os.listdir("data\\fasta_file\\output")
filelist = sorted(filelist,reverse=True)
for item in filelist:
    result = (re.search(pattern,item))
    temp = item[result.start():result.end()]
    if temp in outfilelist:
        pass
    else:
        path = "data\\fasta_file\\output\\" + item
        outpath = "data\\blast_result\\output\\" + item[:-4]+ "_out.xml"
        try:
            Blast_fasta(path,outpath)
            print(item + " Blastn success!")
            print("-"*10)
        except:
            pass

LWW016_all_86.fas Blastn success!
----------


In [42]:
#read xml information

result_dict = {}
temp_query = ""
count = 0


import xml.etree.ElementTree as  ET
import pandas as pd
import os

filelist = os.listdir("data\\blast_result\\output")
filelist = sorted(filelist, reverse=True)

for i in filelist:
    path = "data\\blast_result\\output\\" + i
    for event, item in ET.iterparse(path,events=("start", "end")):
        if item.tag == 'Iteration_query-def' and event == "start":
            temp_query = item.text
        if item.tag == 'Hit' and event == "start":
            try:
                if item[0].text == '1':
                    count+=1
                    try:
                        pattern = "[\w]+ [\w]+"
                        reres = re.search(pattern, item[2].text)
                        result_dict[temp_query] = [item[2].text[reres.start():reres.end()].replace(" ", "_")] #sci_name
                    except:
                        pass
                    try:
                        result_dict[temp_query].append(item[3].text) #accession
                    except:
                        pass
                    try:
                        result_dict[temp_query].append(f"{(int(item[5][0][10].text)/int(item[5][0][13].text))*100}") #identity

                        
                    except:
                        pass
            except:
                # print("query " + str(count) + " error")
                count+=1
                pass

In [48]:
#read original all file and modify info
path = "data\\fasta_file\\LWW016_all.fas"

def read_all_file(path):
    with open (path, "r") as obj:
        txt = obj.read()
    return txt    

txt = read_all_file(path)
abc=0
import re

for item in result_dict:
    pattern = item
    try:
        txt = re.sub(pattern, result_dict[item][0] + "_" + result_dict[item][1] + "_" + result_dict[item][2] + "_" + item, txt)
    except:
        pass

In [36]:
#write output to fasta

with open ("all_output.fas","w") as obj:
    obj.write(txt)